In [ ]:
!poetry add $(cat requirements.txt)

## COLLECT DATA


In [1]:
from binance.client import Client
from tqdm import tqdm

client = Client()
exchange_info = client.get_exchange_info() # get exchange info

print(f"market info: {len(exchange_info['symbols'])}")


market info: 2873


In [2]:
tickers = client.get_ticker()
print(f"all tickers: {len(tickers)}")

ticker_lt = {t['symbol']: t for t in tickers}
print(f"ticker lookup table: {ticker_lt}")

all tickers: 2876
ticker lookup table: {'ETHBTC': {'symbol': 'ETHBTC', 'priceChange': '0.00124000', 'priceChangePercent': '3.460', 'weightedAvgPrice': '0.03655910', 'prevClosePrice': '0.03584000', 'lastPrice': '0.03708000', 'lastQty': '1.20460000', 'bidPrice': '0.03707000', 'bidQty': '61.89250000', 'askPrice': '0.03708000', 'askQty': '2.81230000', 'openPrice': '0.03584000', 'highPrice': '0.03719000', 'lowPrice': '0.03582000', 'volume': '24612.81750000', 'quoteVolume': '899.82251157', 'openTime': 1735908221298, 'closeTime': 1735994621298, 'firstId': 482586857, 'lastId': 482663464, 'count': 76608}, 'LTCBTC': {'symbol': 'LTCBTC', 'priceChange': '0.00002300', 'priceChangePercent': '2.057', 'weightedAvgPrice': '0.00113845', 'prevClosePrice': '0.00111800', 'lastPrice': '0.00114100', 'lastQty': '1.49800000', 'bidPrice': '0.00114000', 'bidQty': '459.36600000', 'askPrice': '0.00114100', 'askQty': '75.41700000', 'openPrice': '0.00111800', 'highPrice': '0.00116700', 'lowPrice': '0.00111100', 'vol

In [14]:
MARKET_FILTERS = {
    'MAX_DAYS': 90,
    'MIN_VOLUME': 50_000,       # $50K daily volume (minimum liquidity)
    'MAX_VOLUME': 10_000_000,   # $10M daily volume (not too liquid)
    'MIN_MARKET_CAP': 1_000_000,  # $1M market cap (avoid extreme micro caps)
    'MAX_MARKET_CAP': 100_000_000  # $100M market cap (true small cap ceiling)
}

In [13]:
usdt_pairs = []
for symbol_info in tqdm(exchange_info['symbols'], desc="Filtering USDT pairs"):
    symbol = symbol_info['symbol']
    
    if not (symbol.endswith('USDT') and symbol_info['status'] == 'TRADING'):
        continue
        
    ticker = ticker_lt.get(symbol)
    if not ticker:
        continue
        
    volume_24h = float(ticker.get('quoteVolume', 0))
    if not (MARKET_FILTERS['MIN_VOLUME'] <= volume_24h <= MARKET_FILTERS['MAX_VOLUME']):
        continue
        
    usdt_pairs.append({
        'symbol': symbol,
        'volume_24h': volume_24h
    })

print(f"Found {len(usdt_pairs)} pairs meeting volume criteria")

Filtering USDT pairs: 100%|██████████| 2873/2873 [00:00<00:00, 80106.33it/s]

Found 255 pairs meeting volume criteria


In [18]:
from joblib import Memory
import os

In [17]:
cache_dir = './.cache'
os.makedirs(cache_dir, exist_ok=True)
memory = Memory(cache_dir, verbose=0)

@memory.cache
def get_klines_cached(symbol, interval, limit):
    return client.get_klines(symbol=symbol, interval=interval, limit=limit)

candidates = []
for pair in tqdm(usdt_pairs, desc="Checking age and market cap"):
    symbol = pair['symbol']
    try:
        klines = get_klines_cached(symbol, interval='1d', limit=90)
        days_listed = len(klines)
        
        if days_listed >= 90:  
            continue
                        
        ticker = ticker_lt[symbol]
        market_cap = float(ticker.get('marketCap', float(ticker['quoteVolume']) * 7))
        if not (MARKET_FILTERS['MIN_MARKET_CAP'] <= market_cap <= MARKET_FILTERS['MAX_MARKET_CAP']):
            continue
            
        candidates.append({
            'symbol': symbol,
            'days_listed': days_listed,
            'market_cap': market_cap,
            'volume_24h': pair['volume_24h']
        })
        
        print(f"Found candidate: {symbol} (MCap: ${market_cap:,.2f}, Age: {days_listed} days)")
        
    except Exception as e:
        print(f"Error processing {symbol}: {str(e)}")
        continue

Checking age and market cap:  99%|█████████▉| 252/255 [02:10<00:01,  1.92it/s]

Found candidate: BNSOLUSDT (MCap: $32,588,419.07, Age: 80 days)


Checking age and market cap:  99%|█████████▉| 253/255 [02:12<00:01,  1.50it/s]

Found candidate: LUMIAUSDT (MCap: $56,122,133.65, Age: 79 days)


Checking age and market cap: 100%|█████████▉| 254/255 [02:12<00:00,  1.32it/s]

Found candidate: KAIAUSDT (MCap: $58,542,015.34, Age: 66 days)


Checking age and market cap: 100%|██████████| 255/255 [02:13<00:00,  1.91it/s]

Found candidate: ORCAUSDT (MCap: $31,880,200.57, Age: 30 days)
